In [1]:
import pandas as pd
import networkx as nx
import itertools

In [2]:
df = pd.read_json("rawfilteredbydate.json")

In [3]:
df.head()

,caption,date
0,The Perlman Music Program Class of 2014,2014-08-22
1,Toby Perlman addresses the crowd,2014-08-22
2,Itzhak Perlman and Catherine Cho perform with...,2014-08-22
3,"Julian Beck, Lyn Barlow, Ted Ayala, Elinor He...",2014-08-22
4,Toby Perlman and guests,2014-08-22


In [4]:
df["caption"] = df['caption'].apply(lambda x: x.replace("\n",""))

In [5]:
df = df[df['caption'].apply(lambda x: len(x)<=250)]

In [22]:
restr = ["The", "Dr", "Verdana", "Helvetica" ,"Chairman","Director",
         "Guests","President","Hospital","Special Surgery","Executive Director"
         ,"New York","Museum","Photographs","Children","The Society","Dinner",
        "Board Member","Castle"]

In [92]:
def replaceWord(s):
    for i in range(len(restr)):
        s = s.replace(restr[i],"")
    s = s.replace("and",",")
    return s

In [9]:
proper_noun_regex = r'([A-Z][A-Za-z-]*(,?\s+(\b(and|von|de|van der|van|der)\b))?(\s[A-Z][A-Za-z\'-]*)*)'

In [10]:
import re

In [23]:
G=nx.MultiGraph()
for caption in df.caption:
    name = []
    caption_temp  = re.findall(proper_noun_regex,caption)
    for j in range(len(caption_temp)):
        name_temp = caption_temp[j]
        if len(name_temp[0])>5:
            if name_temp[0] == "PatrickMcMullan":
                name.append("Patrick McMullan")
            else:
                name.append(name_temp[0])
        for word in restr:
            if word in name:
                name.remove(word)
    if len(name) > 2:
        _=[G.add_edge(c[0],c[1],weight=1) for c in itertools.combinations(name,2)]
    elif len(name) > 1:
        G.add_edge(name[0],name[1],weight=1)        

In [24]:
Gg = nx.Graph()
for u,v,data in G.edges_iter(data=True):
    if Gg.has_edge(u,v):
        Gg[u][v]['weight'] += 1
    else:
        Gg.add_edge(u, v, weight=1)

## degrees

In [25]:
degrees=pd.DataFrame(G.degree().values(),index=G.degree().keys()).sort(0,ascending=False)

In [26]:
degrees = degrees.reset_index()

In [27]:
degrees.columns = ["name","number"]

In [28]:
degrees[0:100]

,name,number
0,Patrick McMullan,2111
1,Jean Shafiroff,689
2,Mark Gilbertson,537
3,Gillian Miniter,463
4,Geoffrey Bradfield,418
5,Alexandra Lebenthal,399
6,Rob Rich,368
7,Andrew Saffir,337
8,Cutty McGill,332
9,Sharon Bush,331


In [18]:
degreeres = map(lambda x,y: (x,y),degrees.name[0:100], degrees.number[0:100])

In [19]:
import pickle

In [20]:
with open('degreeres.pickle', 'wb') as f:
    pickle.dump(degreeres, f)

In [14]:
pr=nx.pagerank(Gg)
pranalysis=pd.DataFrame(pr.values(),index=pr.keys()).sort(0,ascending=False)